In [1]:
from TopicSegmentation import LegalBert, ModifiedStandardDecoder, PaddingMaskLayer
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import re
from Preprocessing import *

# Instantiate Model Architecture

In [2]:
# Model Configuration
vocab_size = 31000
embedding_dim = 768
num_heads = 8
ff_dim = 1024
dropout_rate = 0.1

decoder = ModifiedStandardDecoder(vocab_size, embedding_dim, num_heads, ff_dim, dropout_rate)

# Inputs to the model
segment_inputs = tf.keras.Input(shape=(None,), name="Segment_Input_Layer") # refers to the tokens fed into one at a time
encoder_outputs = tf.keras.Input(shape=(None, embedding_dim), name="Context_Vector_Input_Layer")

padding_mask_layer = PaddingMaskLayer(num_heads=num_heads, name="Padding_Mask_Layer")
padding_mask = padding_mask_layer(segment_inputs)

outputs = decoder(segment_inputs, encoder_outputs, padding_mask=padding_mask)
model_issues = tf.keras.Model(inputs=[encoder_outputs, segment_inputs], outputs=outputs)

model_issues.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Segment_Input_Layer           │ (None, None)              │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ Context_Vector_Input_Layer    │ (None, None, 768)         │               0 │ -                          │
│ (InputLayer)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ modified_standard_decoder     │ (None, None, 31000)       │      87,010,328 │ Segment_Input_Layer[0][0], │
│ (ModifiedStandardDecoder)     │                           │                 │ Context_Vector_Input_Laye… │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 87,010,328 (331.92 MB)

 Trainable params: 87,010,328 (331.92 MB)

 Non-trainable params: 0 (0.00 B)

# Preprocess Data

In [3]:
x = preprocess("new_court_cases.csv")
court_cases = x.give_courtcases()
facts = x.give_rfi()

shape of initial dataframe:  (325, 4)
shape of dataframe when null values were dropped:  (325, 4)
shape of dataframe when preprocessed and duplicated values where dropped:  (325, 4)


In [6]:
court_cases = court_cases[:10]
facts = facts[:10]

# Instantiate Legal BERT Preprocessor and Encoder

In [7]:
# Initialize the preprocessor and legal BERT
legal_bert = LegalBert()

# Get context vectors of the whole court cases in batches
bert_output = legal_bert.get_context_vectors(court_cases, batch_size=32)

C:\Users\mdfl0\AppData\Roaming\Python\Python311\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
bert_output.shape

torch.Size([10, 1024, 768])

# Tokenize and add paddings to each data for each model

In [10]:
tokenizer_issues = legal_bert.tokenizer
# Tokenize the issues segments using LegalBERT tokenizer
tokenized_segments_issues = tokenizer_issues(facts, padding=True, truncation=True, max_length=1024, return_tensors='tf')

# Convert to tensors
input_ids = tokenized_segments_issues['input_ids']

# Shift the input sequences to the right by one position
shifted_segments_issues = np.zeros_like(input_ids.numpy())
shifted_segments_issues[:, 1:] = input_ids[:, :-1]  # Shift right
shifted_segments_issues[:, 0] = tokenizer_issues.cls_token_id  # Use BERT's [CLS] token ID as the start token

# Convert shifted sequences to tensor
shifted_segments_issues = tf.convert_to_tensor(shifted_segments_issues)

# Set the court cases to xtrain
xbert_train = bert_output.detach().numpy()
xtrain = tf.convert_to_tensor(xbert_train, dtype=tf.float32)

# Set the segments to the ytrain and ytrain shifted
ytrain_issues = tf.convert_to_tensor(input_ids, dtype=tf.int32)
ytrain_shifted = tf.convert_to_tensor(shifted_segments_issues, dtype=tf.int32)

In [11]:
print("xtrain shape:", xtrain.shape)  # Should be (batch_size, seq_len, embedding_dim)
print("ytrain_shifted shape:", ytrain_shifted.shape)  # Should be (batch_size, seq_len)
print("ytrain_issues shape:", ytrain_issues.shape)  # Should be (batch_size, seq_len)

xtrain shape: (10, 1024, 768)
ytrain_shifted shape: (10, 1024)
ytrain_issues shape: (10, 1024)


# Train Models

In [12]:
# Compile the models
model_issues.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_issues.fit([xtrain, ytrain_issues], ytrain_shifted, epochs=3)
model_issues.save('facts_seq_to_seq.keras')

Epoch 1/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.0000e+00 - loss: 16.3123
Epoch 2/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 17s 17s/step - accuracy: 0.1206 - loss: 13.3118
Epoch 3/3
1/1 ━━━━━━━━━━━━━━━━━━━━ 13s 13s/step - accuracy: 0.1216 - loss: 11.1890


In [14]:
# Tokenize using BERT's start and end tokens
start_token_id = tokenizer.cls_token_id  # BERT's [CLS] token ID
end_token_id = tokenizer.sep_token_id    # BERT's [SEP] token ID

# Example segment text
segment_text = "The car, a sleek black sedan, purred down the deserted street. Inside, the killer, a cold, calculating figure, gripped the steering wheel tightly."

# Encode segment text without adding special tokens
segment_token_ids = tokenizer.encode(segment_text, add_special_tokens=False)

# Create decoder input with only one [CLS] token at the start
decoder_input = [start_token_id] + segment_token_ids

# Create decoder output with only one [SEP] token at the end
decoder_output_shifted = segment_token_ids + [end_token_id]

# Print to verify
print("Decoder Input (Token IDs):", decoder_input)
print("Decoder Output Shifted (Token IDs):", decoder_output_shifted)

Decoder Input (Token IDs): [101, 207, 1645, 115, 145, 163, 21076, 2709, 28216, 184, 115, 6334, 7889, 476, 207, 15022, 252, 1873, 117, 4867, 115, 207, 27016, 115, 145, 4510, 115, 3075, 3283, 115, 26990, 252, 207, 5123, 235, 5041, 12003, 301, 117]
Decoder Output Shifted (Token IDs): [207, 1645, 115, 145, 163, 21076, 2709, 28216, 184, 115, 6334, 7889, 476, 207, 15022, 252, 1873, 117, 4867, 115, 207, 27016, 115, 145, 4510, 115, 3075, 3283, 115, 26990, 252, 207, 5123, 235, 5041, 12003, 301, 117, 102]


In [16]:
ytrain_issues

<tf.Tensor: shape=(10, 1024), dtype=int32, numpy=
array([[  101, 21351,   189, ...,   434,   249,   102],
       [  101,   226,   223, ...,   117,   221,   102],
       [  101,   207,   889, ...,  9839,   207,   102],
       ...,
       [  101,   226,   223, ...,     0,     0,     0],
       [  101,   226,   223, ...,   190,   189,   102],
       [  101,   151,   117, ...,   210, 24240,   102]])>

In [17]:
shifted_segments_issues

<tf.Tensor: shape=(10, 1024), dtype=int32, numpy=
array([[  101,   101, 21351, ...,   791,   434,   249],
       [  101,   101,   226, ...,   185,   117,   221],
       [  101,   101,   207, ...,   115,  9839,   207],
       ...,
       [  101,   101,   226, ...,     0,     0,     0],
       [  101,   101,   226, ..., 20883,   190,   189],
       [  101,   101,   151, ...,   432,   210, 24240]])>

In [19]:
long_txt = "section 258 of republic act ( r . a .) no . 7160 , also known as the local government code of 1991 ( lgc ), requires the treasurer of the local government to send the warrant of levy to the delinquent owner of the real property , among others . the term “ delinquent owner ” shall be construed as the person registered as owner of the realty based on the certificate of title , and not on the tax declaration . the failure of the treasurer to send the required notices to the delinquent registered owner of the property shall render void not only the levy , but the consequent public auction and sale of the subject property . this appeal by certiorari filed by the city government of antipolo and the city treasurer of antipolo ( city treasurer ; collectively , petitioners ), seek to reverse and set aside the november 18 , 2016 decision and the october 2 , 2017 order of the regional trial court of antipolo city , branch 99 ( rtc ) in civil case no . 14 - 10486 [ 6 ] which declared the forfeiture proceedings conducted by the city treasurer as void . antecedents clarisa san juan santos ( santos ) originally owned three parcels of land located in antipolo city , particularly described as follows : lot : 1 size ( sqm ): 785 tax declaration notice / arpn : aa - 012 - 01841 tct no .: 175110 lot : 2 size ( sqm ): 3 , 350 tax declaration notice / arpn : aa - 012 - 01941 tct no .: 175111 lot : 3 size ( sqm ): 3 , 501 tax declaration notice / arpn : aa - 012 - 01943 tct no .: 175112 all the three tax declarations indicated santos ’ address as “ 3rd st . concepcion [,] marikina city .” sometime in january 1997 , transmix builders & construction , inc . ( respondent ), bought the three lots from santos . consequently , the titles under santos ’ name were cancelled , and new titles were issued in favor of respondent , thus : lot : 1 from tct no .: 175110 to tct no .: 328857 lot : 2 from tct no .: 175111 to tct no .: 328858 lot : 3 from tct no .: 175112 to tct no .: 328859 it appears however , that despite having been registered as new owner of the subject lots , respondent failed to transfer the corresponding tax declarations under its name . on october 30 , 2005 , the city treasurer published in the philippine star , a notice of delinquency of real properties , covering several properties with unpaid real property taxes ( rpt ) located in antipolo city , which included the three lots owned by respondent . the city treasurer also sent the corresponding notices of levy dated november 11 , 2005 to the city assessor and the register of deeds . on november 21 , 2005 , the city treasurer issued warrants of levy over respondent ’ s properties , and sent them to santos at her given address . thereafter , the city treasurer published a notice of public auction sale of real property in the philippine star on november 27 , 2005 . the auction sale was held on december 28 , 2005 , and due to want of bidders , the subject properties were eventually forfeited in favor of the city government of antipolo . consequently , the city treasurer issued a declaration of forfeiture of delinquent real property for each of the three lots on december 28 , 2005 . on february 26 , 2009 , respondent ’ s president , leodegario r . santos , wrote the city treasurer requesting for an opportunity to settle and pay the delinquencies . he claimed that they were unaware of the assessments and statements of account sent by the city treasurer to santos . on july 12 , 2010 , the city government of antipolo passed city ordinance no . 2010 - 398 ( city ordinance ) entitled “ an ordinance prescribing the date of payment without interest of delinquent realty taxes in the city of antipolo ,” the pertinent portions of which read : section 1 . realty tax payment without interest . real property taxes due but unpaid on or before december 31 , 2009 shall not earn interest provided the basic real property taxes and the additional tax for the special education fund are fully paid to the city . section 2 . condition for the availment under section 1 . the payment of real property tax be updated to december 31 , 2010 . section 3 . date of payment . the benefit granted under section 1 maybe availed of not later than december 31 , 2010 . section 4 . property not included in the payment of delinquent realty tax without interest . the benefit granted to delinquent realty tax payment shall not apply to any of the following real properties : a . real properties subject to pending cases in court for real property tax [ d ] elinquencies ; b . real properties , the payment of tax delinquencies of which are the subject of existing compromise agreement ; [ and ] c . real properties which have been disposed of at public auction to satisfy the real property tax and delinquencies . the city treasurer issued on october 19 , 2010 , a notice of real property tax delinquency for the years 1997 to 2010 for each of the three lots , and sent them to santos . the notices bore the address , “ c / o transmix builders & const inc . 2 sumulong highway mayamot antipolo city ,” which appears to be the address indicated in the tax declarations at that time . thus , on november 17 , 2010 , respondent updated and settled the rpt due on the three lots as indicated in the notices . the office of the city treasurer issued the corresponding certifications on january 27 , 2011 , indicating the amounts paid by respondent representing the rpt due on the subject properties . however , the office of the city treasurer , then headed by josefina o . de jesus , sent a letter to respondent dated february 17 , 2011 , stating that since the properties were already forfeited by the city government of antipolo , the payments made by respondent “ will be held in trust until a resolution by a competent authority has been reached .“ the subject parcels of land were eventually registered under the name of the city government of antipolo on march 28 , 2011 . correspondingly , the titles and tax declarations were cancelled and new ones were issued in the name of the city government of antipolo , viz .: lot : 1 tct no . from : 328857 to : r - 79364 tax declaration notice / arpn from : aa - 012 - 01841 to : ac - 012 - 20065 lot : 2 tct no . from : 328858 to : r - 79362 tax declaration notice / arpn from : aa - 012 - 01941 to : ac - 012 - 20063 lot : 3 tct no . from : 328859 to : r - 79363 tax declaration notice / arpn from : aa - 012 - 01943 to : ac - 012 - 20064 aggrieved , respondent filed a complaint for the declaration of nullity of public auction , certificate of sale , titles , and / or reconveyance against the city government of antipolo , the city treasurer and the register of deeds of antipolo city on december 18 , 2014 . respondent alleged that the levy , sale , and eventual forfeiture of the parcels of land were void for lack of notice . ruling of the rtc in its november 18 , 2016 decision , the rtc held that while petitioners “ adhered to the guidelines set forth by law on the forfeiture of a delinquent real property ,” the offer and grant of tax amnesty pursuant to the city ordinance and respondent ’ s subsequent payment of the rpt effectively condoned the tax delinquency . the rtc , thus , ordered the reconveyance of the properties to respondent , viz .: wherefore , premises considered , defendant city government of antipolo is hereby ordered to reconvey unto plaintiff transmix builders & construction , inc . the real properties subject matter hereof , particularly covered by transfer certificates of title nos . r - 79362 , r - 79363 and r - 79364 of the registry of deeds [ of ] antipolo city . the parties ’ claim and counterclaim for damages are both denied . x x x so ordered . unsatisfied , both parties moved for reconsideration . in its october 2 , 2017 order , the rtc modified its earlier decision in the following manner : wherefore , premises considered , defendants ’ motion for reconsideration dated may 25 , 2017 is denied . on the other hand , plaintiff ’ s motion for partial reconsideration dated june 7 , 2017 is granted by amending the decision dated november 18 , 2016 such that the forfeiture proceedings conducted by defendant city treasurer of antipolo on december 28 , 2005 over the real properties covered by tax declaration nos . aa - 012 - 01841 , aa - 012 - 01941 and aa - 012 - 01943 of the city assessor of antipolo , corresponding to tct nos . 328857 , 328858 and 328859 , respectively , are nullified . defendant [ register ] of deeds [ of ] the city of antipolo is directed to cancel tct nos . r - 79362 , r - 79363 and r - 79364 in the name of the city government of antipolo and to reinstate tct nos . 328857 , 328858 and 328859 , in the name of transmix builders and construction , inc . in the same light , the clerk of court of the regional trial court of antipolo city is directed to return unto plaintiff the amount of php7 , 787 , 459 . 36 which the latter deposited on february 9 , 2015 per official receipt no . 7774427 of even date . so ordered . the rtc noted that since the public auction conducted by the city government of antipolo was held in 2005 , the governing law should be the lgc . in this light , it rejected petitioners ’ reliance on the ruling in estate of jacob v . court of appeals ( jacob ) since the auction therein was governed by presidential decree ( p . d .) no . 464 . citing sec . 260 of the lgc and sarmiento v . court of appeals ( sarmiento ), the trial court explained that notice to the delinquent taxpayer was essential to the due process requirement . being an in personam proceeding , the rtc declared that actual notice to the delinquent taxpayer should have been sent to respondent as the registered owner of the subject properties . since no notice had been mailed or served to respondent , the forfeiture was void . due to the void auction proceedings , the rtc found as proper to return the deposit made by respondent in the amount of p7 , 787 , 459 . 36 since the amnesty amount of p2 , 769 , 460 . 00 , covering realty taxes on the subject properties for 2010 and previous years , had already been paid by respondent . aggrieved , petitioners directly appealed to this court to challenge the said order of the rtc . issues the court shall resolve the following issues : ( 1 ) whether the levy , sale , and forfeiture of the subject properties were valid ; ( 2 ) whether petitioners are estopped from accepting rpt payments from respondent ; and ( 3 ) whether the deposit made by respondent should be returned . petitioners ’ arguments petitioners put forth the argument that condonation of rpt could not be granted in favor of respondent . first , pursuant to the lgc , condonation of rpt should only occur when the sangguniang panlungsod enacts an ordinance specifically condoning such rpt , and only in cases of general failure of crops , substantial decrease in the price of agricultural or agri - based products , or calamity in the city , coupled with a recommendation of the local disaster coordinating council . none of the foregoing are present in this case . second , applying the city ordinance in respondent ’ s favor would violate sec . 276 of the lgc . petitioners posit that under sec . 276 , condonation of rpt is applied prospectively , such that the condonation shall take effect on the succeeding year or years following the effectivity of the ordinance . respondent cannot validly avail of the amnesty under the city ordinance without contravening sec . 276 since it would be retroactively applied to the tax delinquencies which accrued prior to its enactment . third , the city ordinance did not effectively condone respondent ’ s tax delinquency , but merely provided tax delinquents with amnesty from interest . under paragraph 3 , sec . 4 of the city ordinance , properties disposed during public auction to satisfy rpt and delinquencies are not included . fourth , petitioners are not estopped from accepting respondent ’ s belated payments . the state cannot be estopped by the mistakes or errors of its officials or agents , especially in the absence of proof that it dealt capriciously or dishonorably with its citizens . fifth , the city treasurer cannot be faulted for not sending notices to respondent because under sec . 73 of p . d . no . 464 , the treasurer is only required to send notices either at the address shown in the tax rolls or property tax record cards of the municipality or city where the property is located , or at the registered owner ’ s residence , if known to such treasurer or barrio captain . since the tax declarations still reflected santos as the registered owner , then the city treasurer had correctly sent the notices to her . finally , should the levy , sale , and forfeiture be held null and void , the city government of antipolo is still entitled to the deposit to cover for the unpaid taxes on the land . respondent ’ s arguments respondent counters in its comment that the instant petition should be dismissed as it raises both factual and legal issues . it likewise contends that the presumption of regularity does not apply when taxpayers are deprived of their properties . petitioners likewise cannot dispense of their obligation to send the notices to respondent being the registered owner , most especially since the subject properties were registered to respondent under the torrens system . moreover , petitioners would unjustly be enriched by the forfeiture considering that the tax delinquency only amounted to p2 , 448 , 887 . 19 , while the three lots have an estimated value of p90 , 000 , 000 . 00 . finally , respondent posits that local government units do not enjoy immunity from suit , and therefore cannot benefit from the principle of estoppel . the court ’ s ruling the appeal has no merit . before resolving the substantive issues raised in this case , we first rule on the propriety of the direct appeal before the court . for a petition for review on certiorari under rule 45 of the rules of court to prosper and warrant the attention of the court , it must satisfy the basic procedural requisites provided in the rules . one of the most basic of all the requirements is that the petition must raise only pure questions of law , otherwise , the petition may be denied outright and without any further action by the court . in mandaue realty & resources corporation v . court of appeals , the court outlined the rules regarding appeals , of cases decided by the rtc : ( 1 ) in all cases decided by the rtc in the exercise of its original jurisdiction , appeal may be made to the court of appeals by mere notice of appeal where the appellant raises questions of fact or mixed questions of fact and law ; ( 2 ) in all cases decided by the rtc in the exercise of its original jurisdiction where the appellant raises only questions of law , the appeal must be taken to the supreme court on a petition for review on certiorari under rule 45 [;] [ and ] ( 3 ) all appeals from judgments rendered by the rtc in the exercise of its appellate jurisdiction , regardless of whether the appellant raises questions of fact , questions of law , or mixed questions of fact and law , shall be brought to the court of appeals by filing a petition for review under rule 42 . ( emphasis and underscoring supplied ) clearly , a direct appeal from the rtc to this court is warranted only when ( 1 ) the rtc exercised its original jurisdiction , and ( 2 ) only questions of law are being raised . the court finds that both requirements were satisfied in the case at bar . evident herein that the rtc rendered the assailed decision and order in the exercise of its original jurisdiction , and that the instant petition raises pure questions of law . it is a settled rule that a question of law exists when there is doubt or controversy as to what the law is on a certain state of facts . on the other hand , there is a question of fact when the doubt or contrast arises as to the truth or falsehood of facts , or when the contention necessarily invites calibration of the whole evidence considering mainly the credibility of witnesses , existence and relevancy of specific surrounding circumstances , their relation to each other and to the whole , and probabilities of the situation . in this case , the issues concerning the validity of the levy , the application of estoppel due to petitioners ’ receipt of payment from respondent , and the entitlement of the city government of antipolo to the deposit made by respondent , involve pure questions of law that do not require an assessment or evaluation of the evidence to resolve the questions posed . in settling these matters , the court is not compelled to recalibrate the body of evidence presented by the parties in the rtc to determine its truth or falsity , as well as its probative value . having settled the procedural issue , the court now proceeds to resolve the substantial matters being disputed by the parties . the levy , sale , and subsequent forfeiture of the subject properties were void ; sec . 258 of the lgc requires sending the notice of levy to the registered owner . petitioners insist on the propriety of sending the required notices to santos being the registered owner based on the tax declarations . on the other hand , respondent contends that since the subject properties were registered under the torrens system , petitioners are under constructive notice of the fact of registration . hence , notices should have been sent to the registered owner based on the transfer certificates of title ( tct ). the court agrees with respondent . unlike land registration proceedings which are in rem , auction sale of land for delinquency of rpt is in personam . contrary to in rem proceedings where publication and posting as a form of notice would suffice , mere publication and posting are insufficient notices in tax sales . simply stated , personal notice to the taxpayer is essential in administrative proceedings of forfeiture of real property . as early as 1908 , this principle of mandatory personal notice to the taxpayer had been established in valencia v . jimenez ( valencia ). in valencia , the court held that notice to the taxpayer is part of due process , thus : the american law does not create a presumption of the regularity of any administrative action which results in depriving a citizen or taxpayer of his property , but , on the contrary , the due process of law to be followed in tax proceedings must be established by proof and the general rule is that the purchaser of a tax title is bound to take upon himself the burden of showing the regularity of all proceedings leading up to the sale . the difficulty of supplying such proof has frequently lead to efforts on the part of legislatures to avoid it by providing by statute that a tax deed shall be deemed either conclusive or presumptive proof of such regularity . the same principle has since been echoed by the court in a plethora of decisions . in the case at bar , it is undisputed that the city treasurer complied with the posting and publication requirements pursuant to sec . 254 of the lgc , which reads , thus : section 254 . notice of delinquency in the payment of the real property tax . — ( a ) when the real property tax or any other tax imposed under this title becomes delinquent , the provincial , city or municipal treasurer shall immediately cause a notice of the delinquency to be posted at the main entrance of the provincial capitol , or city or municipal hall and in a publicly accessible and conspicuous place in each barangay of the local government unit concerned . the notice of delinquency shall also be published once a week for two ( 2 ) consecutive weeks , in a newspaper of general circulation in the province , city , or municipality . ( b ) such notice shall specify the date upon which the tax became delinquent and shall state that personal property may be distrained to effect payment . it shall likewise state that at any time before the distraint of personal property , payment of the tax with surcharges , interests and penalties may be made in accordance with the next following section , and unless the tax , surcharges and penalties are paid before the expiration of the year for which the tax is due , except when the notice of assessment or special levy is contested administratively or judicially pursuant to the provisions of chapter 3 , title ii , book ii of this code , the delinquent real property will be sold at public auction , and the title to the property will be vested in the purchaser , subject , however , to the right of the delinquent owner of the property or any person having legal interest therein to redeem the property within one ( 1 ) year from the date of sale . ( emphases and underscoring supplied ) the dispute , however , concerns the recipient of the corresponding notices which petitioners admit to have not served , either by mail or personal service , upon respondent as the registered owner based on the tcts . for petitioners , the notification requirement has been satisfied when they sent the notices to santos , the previous owner , who , indisputably , is the registered owner based on the tax declarations . the city treasurer insists that she has no duty to look beyond the tax declaration receipts in identifying the true owner of the real properties . she further maintains that it was incumbent upon respondent , as the registered owner , to declare the property and pay the tax concomitant thereto . respondent failed in both aspects . on the other hand , respondent maintains that despite its failure to declare the subject properties , it is the duty of the city treasurer to verify the identity of the registered owner based on the tct , as it is actually the real taxpayer . a scrutiny of the provision on whom to serve the notice , as well as its legislative history , will do well to enlighten us on this matter . prior to the lgc , the law on levy of real property was governed by p . d . no . 464 . the pertinent provision on the notice of levy is governed by sec . 73 of p . d . no . 464 , which reads : section 73 . advertisement of sale of real property at public auction . after the expiration of the year for which the tax is due , the provincial or city treasurer shall advertise the sale at public auction of the entire delinquent real property , except real property mentioned in subsection ( a ) of section forty hereof , to satisfy all the taxes and penalties due and the costs of sale . such advertisement shall be made by posting a notice for three consecutive weeks at the main entrance of the provincial building and of all municipal buildings in the province , or at the main entrance of the city or municipal hall in the case of cities , and in a public and conspicuous place in barrio or district wherein the property is situated , in english , spanish and the local dialect commonly used , and by announcement at least three market days at the market by crier , and , in the discretion of the provincial or city treasurer , by publication once a week for three consecutive weeks in a newspaper of general circulation published in the province or city . the notice , publication , and announcement by crier shall state the amount of the taxes , penalties and costs of sale ; the date , hour , and place of sale , the name of the taxpayer against whom the tax was assessed ; and the kind or nature of property and , if land , its approximate areas , lot number , and location stating the street and block number , district or barrio , municipality and the province or city where the property to be sold is situated . copy of the notice shall forthwith be sent either by registered mail or by messenger , or through the barrio captain , ( 1 ) to the delinquent taxpayer , ( a ) at his address as shown in the tax rolls or property tax record cards of the municipality or city where the property is located , ( b ) or at his residence , if known to said treasurer or barrio captain : provided , however , that a return of the proof of service under oath shall be filed by the person making the service with the provincial or city treasurer concerned . ( emphasis and underscoring supplied ) p . d . no . 464 mandates the treasurer to send the notice to the delinquent taxpayers , at their addresses as shown in the tax rolls or property tax record cards of the municipality or city where the property is located , or at their residence , if known . the notice may be served through registered mail , by messenger , or through the barrio captain . when p . d . no . 464 was repealed by the lgc , the relevant portion on whom to send the notice was removed . instead , sec . 258 of the lgc prescribes the persons to whom the warrant of levy shall be issued : section 258 . levy on real property . — after the expiration of the time required to pay the basic real property tax or any other tax levied under this title , real property subject to such tax may be levied upon through the issuance of a warrant on or before , or simultaneously with , the institution of the civil action for the collection of the delinquent tax . the provincial or city treasurer , or a treasurer of a municipality within the metropolitan manila area , as the case may be , when issuing a warrant of levy shall prepare a duly authenticated certificate showing the name of the delinquent owner of the property or person having legal interest therein , the description of the property , the amount of the tax due and the interest thereon . the warrant shall operate with the force of a legal execution throughout the province , city or a municipality within the metropolitan manila area . the warrant shall be mailed to or served upon the delinquent owner of the real property or person having legal interest therein , or in case he is out of the country or cannot be located , the administrator or occupant of the property . at the same time , written notice of the levy with the attached warrant shall be mailed to or served upon the assessor and the registrar of deeds of the province , city or municipality within the metropolitan manila area where the property is located , who shall annotate the levy on the tax declaration and certificate of title of the property , respectively . ( emphasis supplied ) careful attention should be placed on the words used in the law . to put it in simpler terms , p . d . no . 464 , the predecessor of the lgc , used the words “ delinquent taxpayer ,” while the current law , the lgc , employed the phrase “ delinquent owner .” such change in phraseology was intentional . casus omissus pro omisso habendus est . a person , object , or thing omitted from an enumeration in a statute must be held to have been omitted intentionally . the said change emphasized that the notice must be served to the owner , whose name may or may not be reflected in the tax records . the import of these words as used in the two laws plays a significant role in determining as to whom the notice should be served – either to the person reflected in the tax rolls , or strictly to the registered owner . a plain reading of sec . 73 of p . d . no . 464 would suggest that the treasurer can rely solely on the tax rolls or property tax records in mailing or serving the notice , without regard to the certificates of title or any annotation therein . however , the procedure was significantly changed with the enactment of the lgc . while the treasurer has no duty to look beyond the tax records in serving the notice to the taxpayer under p . d . no . 464 , the lgc now in contrast , distinctively requires that the warrant be mailed or served to the owner of the real property or the person having legal interest over the property . in this light , let us dissect the penultimate sentence of the first paragraph of sec . 258 of the lgc which reads : “[ t ] he warrant shall be mailed to or served upon the delinquent owner of the real property or person having legal interest therein , or in case he is out of the country or cannot be located , the administrator or occupant of the property .” clearly under the above - quoted provision , the city treasurer is given the option to either mail or serve the warrant of levy to the following persons : ( 1 ) the delinquent owner of the real property , ( 2 ) the person having legal interest therein , or ( 3 ) the administrator or occupant of the property , in case the owner or the interested person is out of the country or cannot be located . in contrast , as previously discussed , p . d . no . 464 instructed the treasurer to send the notice only to the delinquent taxpayer as shown in the tax records . by providing options other than the taxpayer as shown in the tax records , the lgc has filled in the void under p . d . no . 464 . now , the lgc does not limit the issuance of a warrant of levy to the taxpayer as reflected in the tax rolls . the amendment made by the lgc allowed room for flexibility in issuing the said warrant , by ensuring that notice is given to the appropriate party . the change in the phraseology in the lgc ensures that the rightful person entitled to the notice , who may or may not be the owner or person registered in the title or in the tax declaration , is accorded due process . more significantly , in employing the word “ owner ,” sec . 258 of the lgc has become consistent with the principle of the indefeasibility of torrens title . by preserving the sanctity of the torrens system , everyone dealing with the property registered under the system may safely rely on the correctness of the certificate of title . otherwise , allowing claims disputing the titles under the torrens system would inevitably challenge the integrity of land registration officials , who are ordinarily presumed to have regularly performed their duties . the binding effect of registration as a principle of the torrens system is expressed in sec . 51 of the property registration decree or p . d . no . 1529 , which reads : section 51 . conveyance and other dealings by registered owner . an owner of registered land may convey , mortgage , lease , charge or otherwise deal with the same in accordance with existing laws . he may use such forms , deeds , mortgages , leases or other voluntary instruments as are sufficient in law . but no deed , mortgage , lease or other voluntary [ instruments ], except a will purporting to convey or affect registered land [,] shall take effect as a conveyance or bind the land , but shall operate only as a contract between the parties and as evidence of authority to the register of deeds to make registration . the act of registration shall be the operative act to convey or affect the land insofar as third persons are concerned , and in all cases under this decree , the registration shall be made in the office of the register of deeds for the province or the city where the land lies . ( emphasis supplied ) hence , the torrens system makes no distinction and is obligatory upon the whole world . it is as binding on buyers , as well as on local government treasurers . in relying on the title issued in the name of respondent and not on the tax records , the city treasurer would be preserving the sanctity of the torrens system and consequently , would be upholding the incontrovertible and indefeasible title of respondent that no one , even the city government of antipolo or the city treasurer , could defeat . in administering her function in levying real property , the city treasurer is constrained by the basic principle of indefeasibility of title . as such , the city treasurer cannot excuse her failure to notify respondent by solely relying on sec . 73 of p . d . no . 464 , which had long been repealed by the lgc . it likewise bears emphasis that the city treasurer cannot feign ignorance on the registration of the title under respondent ’ s name . under the torrens system , respondent ’ s title serves as constructive notice upon the city treasurer that the former is the registered owner and is deemed the taxpayer to whom notice and warrant of levy should have been issued . assuming that the city treasurer ’ s failure to send notice to respondent was due to her understanding of the term “ taxpayer ” as used in sec . 73 of p . d . no . 464 , still , this argument must fail . at this juncture , it is important to note that the term “ taxpayer ” as used in p . d . no . 464 is a misnomer . interestingly , despite the use of the term “ taxpayer ” in p . d . no . 464 , the court had been construing the same to actually refer to the registered owner . for instance , in talusan v . tayag ( talusan ), the court categorically declared that personal notice to the registered owner is required under p . d . no . 464 . the court explained that the registered owner is deemed as the taxpayer for purposes of the collection of rpt . hence , in case of an unregistered deed of sale , the purported property owners , although in possession of the realty , cannot be accorded as the taxpayer . the transaction has no binding effect on third persons who are not privy to the same . in resolving the issue as to whom the notice of delinquency shall be sent , the court held that it is only the registered owner who is entitled to a notice of tax delinquency and other proceedings relative to the tax sale . the court ’ s discussion in talusan on the torrens system vis - à - vis levy of real property , bolsters the sanctity of registered titles . according to the court , “ it is the registration of the deed of sale that can validly transfer or convey a person ’ s interest in a property .” tersely put , it is the person who appears on the certificate of title who is deemed as the taxpayer . although p . d . no . 464 employed the word “ taxpayer ,” the law still actually referred to “ owners ” of the real properties . there is actually no distinction between p . d . no . 464 and the lgc as both laws encompass owners of the realty . thus , regardless of which law applies , whether the progenitor p . d . no . 464 or the extant lgc , the definition of “ owner ” holds true for both laws , such that , all notices should be sent to the registered owner and not merely the taxpayer as reflected on the tax records . any confusion that may have been caused by the wording in p . d . no . 464 was subsequently cured with the change in the phraseology in the lgc . ineluctably , since it is respondent who is the registered owner of the subject properties , the city treasurer should have sent the notices to the former pursuant to sec . 258 of the lgc . clearly , it was erroneous for petitioners to insist that the required notices were validly sent to respondent , despite having sent them to santos at the address indicated on the tax records . it must be emphasized that while respondent failed to immediately declare the properties under its name in the tax declarations , such omission cannot validly excuse the city treasurer from absconding from her duties of serving the notices . the duty of a treasurer in ascertaining the identity of the registered owner for purposes of notification had already been emphasized in jacob . in jacob , the court annulled the public auction sale and ordered the register of deeds of quezon city to cancel tct no . 352727 and issue in lieu thereof , a new title in the name of respondent bernardita c . tolentino . it held that the city treasurer should not have simply relied on the tax declaration in notifying the tax - delinquent owner . since the properties were protected under the mantle of the torrens system , it was incumbent upon the city treasurer to ascertain who the real owner of the properties was . the court emphatically declared in said case that it would have been more prudent for the city treasurer to verify from the office of the register of deeds who the registered owner was to determine the real delinquent taxpayer as defined in sec . 73 . the court held that : in ascertaining the identity of the delinquent taxpayer , for purposes of notifying him of his tax delinquency and the prospect of a distraint and auction of his delinquent property , petitioner city treasurer should not have simply relied on the tax declaration . the property being covered by the torrens system , it would have been more prudent for him , which was not difficult to do , to verify from the office of the register of deeds of quezon city where the property is situated and as to who the registered owner was at the time the auction sale was to take place , to determine who the real delinquent taxpayer was within the purview of the third paragraph of sec . 73 . for one who is no longer the lawful owner of the land cannot be considered the “ present registered owner ” because , apparently , he has already lost interest in the property , hence is not expected to defend the property from the sale at auction . the purpose of pd no . 464 is to collect taxes from the delinquent taxpayer and , logically , one who is no longer the owner of the property cannot be considered the delinquent taxpayer . ( emphasis supplied ) the factual circumstances surrounding the present case square with that in jacob . similar to jacob , the city treasurer in this case relied solely on the address in the tax declaration without verifying who the registered owner was based on the tct . as categorically stated by the court in jacob , “ x x x mere compliance by the treasurer with sec . 65 of the decree is no longer enough . the notification to the right person , [ i . e .,] the real owner , is an essential and indispensable requirement of the law , [ noncompliance ] with which renders the auction sale void .” applying the foregoing , sending a notice to the previous owner , santos , despite registration of the properties under the name of respondent , is a fatal mistake on the part of the city treasurer . the city treasurer should have been more prudent and diligent in issuing the required notices . the court is aware that in talusan , the purported owners not only failed to register the deed of sale , but also omitted to consolidate the title in their name . worse , they also failed to pay the rpt due . appropriately , the court held that the owners were barred by laches to question the levy on the property despite the fact that they had been in possession of the property since 1981 . such does not obtain in this case . herein , respondent registered the properties in its name and had been in possession of the same . not to disparage the importance of paying the rpt , the only fault of respondent was to overlook declaring the subject properties for tax purposes , contrary to what had happened in talusan . as such , respondent is under the protection of the torrens system which should have sufficed as constructive notice upon the city treasurer . as lengthily discussed , the city treasurer should have sent the required notices to respondent , being the registered owner of the three lots . the option where to send notices under p . d . no . 464 has become passé with the passage of the lgc . despite this , petitioners insist that mailing of the notice to the taxpayer ’ s address based on tax records is justified by our ruling in aquino v . quezon city ( aquino ). the court is not convinced . first , aquino applied secs . 65 and 73 of p . d . no . 464 , the law then in force . as pointed out earlier , p . d . no . 464 is not applicable in this case as it was repealed by the lgc . second , in aquino , the court held that the treasurer could not be faulted for not sending notices to the owner ’ s , address in quezon city as indicated in the tax declarations . pertinent in aquino was the fact that the real property owner did not declare his correct address in either his title or the tax declarations . it also appeared that he could no longer be found in butuan city as he had moved to quezon city in 1959 . the court noted that the registered owner failed to amend his address for more than 25 years , and in failing to do so , he had become aware of the likelihood that the notices of tax delinquencies would be sent to him at his last known address , which he had tersely indicated as “ butuan city .” apparently , the doctrine in aquino is inapplicable in the case at bar . as already discussed , the option given to the treasurer on where to send notices had clearly been removed by the lgc . while there are provisions in p . d . no . 464 that were not carried over and incorporated in the lgc , the basic maxim that levy proceedings are in personam still applies ; and pursuant to sec . 258 of the lgc , notice should be mailed or served upon the registered owner as a requirement of due process . undeniably , respondent did not come to court with clean hands . respondent , as the registered owner , had the incontrovertible duty to declare and pay the rpt over the subject properties . this responsibility of an owner is inscribed in sec . 202 of r . a . no . 7160 , to wit : section 202 . declaration of real property by the owner or administrator . — it shall be the duty of all persons , natural or juridical , owning or administering real property , including the improvements therein , within a city or municipality , or their duly authorized representative , to prepare , or cause to be prepared , and file with the provincial , city or municipal assessor , a sworn statement declaring the true value of their property , whether previously declared or undeclared , taxable or exempt , which shall be the current and fair market value of the property , as determined by the declarant . such declaration shall contain a description of the property sufficient in detail to enable the assessor or his deputy to identify the same for assessment purposes . the sworn declaration of real property herein referred to shall be filed with the assessor concerned once every three ( 3 ) years during the period from january first ( 1st ) to june thirtieth ( 30th ) commencing with the calendar year 1992 . indeed , respondent is not without fault . nonetheless , the court explained in jacob that the assessor , should the registered owner fail to declare the property , has the corresponding duty to list the real property for taxation . sec . 7 of p . d . no . 464 reads : section 7 . declaration of real property by the assessor . when any person , natural or juridical , by whom real property is required to be declared under section six hereof refuses or fails for any reason to make such declaration within the time prescribed , the provincial or city assessor shall himself declare the property in the name of the defaulting owner , if known , or against an unknown owner , as the case may be , and shall assess the property for taxation in accordance with the provisions of this code . no oath shall be required of a declaration thus made by the provincial or city assessor . although the law applicable in jacob is p . d . no . 464 , the court may still refer to the aforesaid case considering that the principles carried over from p . d . no . 464 , and thereafter adopted in the lgc , remain binding and subsisting . even if the lgc has repealed p . d . no . 464 , it carries a parallel provision on the duty of the assessor in case of refusal or failure on the part of the owner to declare the property . sec . 204 of the lgc reads : section 204 . declaration of real property by the assessor . – when any person , natural or juridical , by whom real property is required to be declared under section 202 hereof , refuses or fails for any reason to make such declaration within the time prescribed , the provincial , city or municipal assessor shall himself declare the property in the name of the defaulting owner , if known , or against an unknown owner , as the case may be , and shall assess the property for taxation in accordance with the provision of this title . no oath shall be required of a declaration thus made by the provincial , city or municipal assessor . this duty of the assessor to declare a property in the name of the defaulting owner , in case the latter refuses or fails to declare the same , is to avoid any error as to whom the notice should be sent to , especially in case there is a change of ownership in the real properties . also , while it is conceded that respondent can be faulted for failure to declare the subject properties under its name , the city treasurer carries the onus probandi that she has complied with the procedure in levying the subject real properties . as already held , local government units do not enjoy presumption of regularity in forfeiture proceedings of real property . the burden of proof is upon the local government to prove that it had religiously complied with the rules . unfortunately , petitioners failed to overcome such onus . these principles of burden of proof upon the taxing authority and non - applicability of presumption of regularity , as declared in cruz v . city of makati ( cruz ), are to favor taxpayers and address the unfortunate truth of possible abuse of the taxing power . the court therein warned of the exploitation of the power to tax and the subsequent authority of the government to levy . it likewise recognized the destructive effect once a taxing authority deprives a taxpayer of a property . it is rather necessary , thus , that due process of law must be religiously complied with . it bears emphasizing that these precepts favoring the taxpayers are meant to prevent the possibility of collusion between buyers and public officials which may result in depriving taxpayers of their property . the rationale behind safeguarding the rights of taxpayers is best explained by the court in sarmiento , thus : we cannot overemphasize that strict adherence to the statutes governing tax sales is imperative not only for the protection of the taxpayers , but also to allay any possible suspicion of collusion between the buyer and the public officials called upon to enforce the laws . notice of sale to the delinquent [ landowners ] and to the public in general is an essential and indispensable requirement of law , the non - fulfillment of which vitiates the sale . thus , the holding of a tax sale despite the absence of the requisite notice is tantamount to a violation of delinquent taxpayer ’ s substantial right to due process . administrative proceedings for the sale of private lands for nonpayment of taxes being in personam , it is essential that there be actual notice to the delinquent taxpayer , otherwise the sale is null and void although preceded by proper advertisement or publication . ( emphasis supplied ) the court cannot turn a blind eye on the fatal defect in the levy proceedings committed by the city treasurer . lest it be misunderstood , such statement is not to promote neglect on taxpayers to declare their properties . it is still incumbent upon registered owners to declare their real properties . in cruz , the court had the occasion to unveil the disheartening anomalies in the levy and sale of real properties due to delinquency in the payment of taxes due to the sending of notices to wrong addresses , to wit : the court must protect private property owners from undue application of the law authorizing the levy and sale of their properties for [ nonpayment ] of the real property tax . this power of local government units is prone to great abuse , in that owners of valuable real property are liable to lose them on account of irregularities committed by these local government units or officials , done intentionally with the collusion of third parties and with the deliberate unscrupulous intent to appropriate these valuable properties for themselves and profit therefrom . these unscrupulous parties can commit a simple , seemingly irrelevant technicality such as deliberately sending billing statements , notices of delinquency and levy to wrong addresses under the guise of typographical lapses , as what happened here and in the genato investments case , and then proceed with the levy and auction sale of these valuable properties without the knowledge and consent of the owners . before the owners realize it , their precious properties have already been confiscated and sold by the local government units or officials to so - called “ innocent third parties ” who are in fact their cohorts in the unscrupulous scheme . this is barefaced robbery that the court cannot sanction . x x x there is a grave danger that taxpayers may unwittingly lose their real properties to unscrupulous local government units , officials , or private individuals or entities as a result of an irregular application of the lgc provisions authorizing the levy and delinquency sale of real property for [ nonpayment ] of the real property tax . this is a reality that cannot be ignored . for this reason , the court must excuse petitioners for their procedural lapses , as it must address instead the issue of irregular conduct of levies and delinquency sales of real properties for [ nonpayment ] of the real property tax , which is alarming considering that of the two cases that this court is made aware of , there appears to be one common denominator , and that is the respondent herein , laverne realty and development corporation . needless to state , petitioners are liable to lose their property without due process of law to laverne which was previously involved in an irregular sale conducted under similar circumstances . ( emphasis supplied ) the predicament that respondent faces in the present case is not far from the evil sought to be prevented by the basic maxim that notice should be mailed or served to the registered owner . it should be stressed that while registered owners have the duty to declare their property for payment of rpt , non - declaration of the same does not do away with the treasurer ’ s responsibility to ascertain the identity of the true registered owner thereof . for accuracy , we refer to the last sentence of the first paragraph of sec . 258 of the lgc on the duty of a treasurer to notify the assessor and the register of deeds of the levy , which reads : “ at the same time , written notice of the levy with the attached warrant shall be mailed to or served upon the assessor and the [ register ] of deeds of the province , city or municipality within the metropolitan manila area where the property is located , who shall annotate the levy on the tax declaration and certificate of title of the property , respectively .” although not found in the repealed p . d . no . 464 , the same provision was incorporated in chapter 4 of the lgc on civil remedies for collection of revenues , specifically sec . 176 , which reads : section 176 . levy on real property . — after the expiration of the time required to pay the delinquent tax , fee , or charge , real property may be levied on before , simultaneously , or after the distraint of personal property belonging to the delinquent taxpayer . to this end , the provincial , city or municipal treasurer , as the case may be , shall prepare a duly authenticated certificate showing the name of the taxpayer and the amount of the tax , fee , or charge , and penalty due from him . said certificate shall operate with the force of a legal execution throughout the philippines . levy shall be effected by writing upon said certificate the description of the property upon which levy is made . at the same time , written notice of the levy shall be mailed to or served upon the assessor and the register of deeds of the province or city where the property is located who shall annotate the levy on the tax declaration and certificate of title of the property , respectively , and the delinquent taxpayer or , if he be absent from the philippines , to his agent or the manager of the business in respect to which the liability arose , or if there be none , to the occupant of the property in question . ( emphasis supplied ) accordingly , the last sentence of the first paragraph of sec . 258 of the lgc instructs the treasurer to send a written notice of the levy with the attached warrant to the assessor and the register of deeds , whose duty is to annotate the levy on the tax declaration and certificate of title . the sending of this notice is simultaneous with the required notice to the registered owner . prescinding from the foregoing and given the lack of notice to respondent as the registered owner , the levy , sale , and consequent forfeiture of the subject lots by the city government of antipolo , are void for lack of due process . respondent properly availed of the amnesty under the city ordinance . petitioners argue that respondent could not have properly availed of the amnesty under the city ordinance because it excludes those properties that were sold on public auction due to tax delinquencies . the court disagrees . sec . 4 of the city ordinance explicitly enumerates the properties excluded from its coverage : section 4 . property not included in the payment of delinquent realty tax without interest . the benefit granted to delinquent realty tax payment shall not apply to any of the following real properties : a . real properties subject to pending cases in court for real property taxes ; b . real properties , the payment of tax delinquencies of which are the subject of an existing compromise agreement ; and c . real properties which have been disposed of at public auction to satisfy the real property tax and delinquencies . ( emphasis supplied ) the instant case does not fall under any of the foregoing categories . as earlier discussed , the levy , auction sale , and consequent forfeiture by the city government of antipolo are void for failure to strictly observe due process . hence , the third exception which petitioners espouse as ground against respondent does not avail . evidently , the subject properties are within the coverage of the city ordinance for which respondent may avail of the amnesty as provided . it must be emphasized that other than the exception under sec . 4 , the city ordinance provides that a taxpayer may avail of the amnesty subject to the following conditions : ( 1 ) the basic rpt and additional tax for the special education fund are fully paid ; ( 2 ) the payment is updated until december 31 , 2010 ; and ( 3 ) the amnesty should be availed not later than december 31 , 2010 . as found by the rtc , respondent had paid petitioners the delinquent rpt for the years 1997 to 2009 based on the assessment issued by the city treasurer , and updated its payment for 2010 as provided under the city ordinance . accordingly , respondent had properly availed of the amnesty on interest under the city ordinance . petitioners also make issue on the ruling of the rtc to return the amount consigned by respondent in the amount of p7 , 787 , 459 . 36 on february 9 , 2015 per official receipt no . 7774427 . however , we find nothing wrong with the said ruling . to recall , respondent consigned the said amount to the rtc pursuant to sec . 267 of the lgc which reads : section 267 . action assailing validity of tax sale . – no court shall entertain any action assailing the validity or any sale at public auction of real property or rights therein under this title until the taxpayer shall have deposited with the court the amount for which the real property was sold , together with interest of two percent ( 2 %) per month from the date of sale to the time of the institution of the action . the amount so deposited shall be paid to the purchaser at the auction sale if the deed is declared invalid but it shall be returned to the depositor if the action fails . neither shall any court declare a sale at public auction invalid by reason or irregularities or informalities in the proceedings unless the substantive rights of the delinquent owner of the real property or the person having legal interest therein have been impaired . evident from the wording of sec . 267 is that the required deposit should be paid only to the purchaser of real property , in this case , the city government of antipolo , should the auction sale be held invalid . it is in the nature of a repurchase by the owner of a property improperly sold to the purchaser . however , such circumstance does not avail in the present case . to recall , the levy , public auction , and forfeiture by the city government of antipolo of the subject properties are hereby declared void for lack of due process . while the void levy and forfeiture originated from the nonpayment of respondent of the rpt on the three lots , the amount deposited by respondent cannot be paid to the city government because , the delinquent rpt for which the subject realty were forfeited , had already been paid by respondent under the amnesty provided by the city ordinance . no error can , thus , be attributed to the rtc for ordering the return of the deposit to respondent because it no longer owes the city government delinquent rpt covering the period 1997 to 2010 . to do otherwise would unjustly enrich the city government which the court cannot allow . before leaving this matter to rest , the court emphasizes that this ruling only covers respondent ’ s rpt liabilities and corresponding payments covering the period 1997 to 2010 . due to the unlawful levy , sale , and subsequent forfeiture by the city government of antipolo , the subject lots are still owned by respondent , and should rightfully be returned to the latter . hence , the tcts issued under the name of the city government of antipolo should be cancelled , and the previous titles in the name of respondent should be reinstated . the state is not subject to estoppel by the mistakes or errors of its officials or agents in the absence of proof that it had dealt capriciously or dishonorably with its citizens . anent the issue on whether petitioners are estopped from levying the subject properties on the ground that they had accepted payments from respondent , petitioners are partly correct . it is a basic concept in law that the state is not subject to estoppel by the mistakes or errors of its officials or agents , especially in the absence of any showing that it had dealt capriciously or dishonorably with its citizens . in the case at bar , while the city treasurer accepted respondent ’ s payment for its tax delinquency , such acceptance cannot hold the city government of antipolo in estoppel . at this juncture , it is important to note that the city treasurer ’ s acceptance of respondent ’ s payment was qualified . in the letter to respondent dated february 17 , 2011 , the city treasurer had explicitly declared that the fee “ will be held in trust until a resolution by a competent authority has been reached .” in fact , to avoid any misinterpretation that the city treasurer or the city government of antipolo had condoned respondent ’ s tax delinquency , the city treasurer declared that respondent may withdraw the amount anytime . it is worthy to emphasize that respondent is fully aware of its transgression of not paying the rpt . acknowledging such mistake , respondent wrote a letter signifying its intent to settle and pay said realty taxes . the pertinent portion of respondent ’ s february 26 , 2009 letter to the city treasurer reads : please be assured that we are willing to settle and pay the realty taxes on the said property . it is our hope that we would be given the opportunity to settle and pay the realty taxes . regardless of whether the city treasurer committed a mistake by sending notices to santos , or accepted payments from respondent , the city treasurer ’ s actions cannot bind its principal , the city government of antipolo . it is important to note that respondent cannot plead innocence . as already held , respondent failed to pay its rpt not just for one taxable year , but for seven long years from 1997 to 2004 . moreover , acceptance of tax payments cannot amount to estoppel which would effectively result in the condonation of a tax liability . in sum , notice of sale to the delinquent landowners and to the public in general is an essential and indispensable requirement of law , the non -¬ fulfillment of which vitiates the sale . absence of the requisite notice amounts to a violation of the delinquent taxpayer ’ s substantial right to due process . strict adherence to the statutes governing tax sales is imperative not only for the protection of the taxpayers , but also to allay any possible suspicion of collusion between the buyer and the public officials called upon to enforce the laws . wherefore , the petition is denied . the november 18 , 2016 decision and the october 2 , 2017 order rendered by the regional trial court of antipolo city , branch 99 in civil case no . 14 - 10486 are affirmed ."

In [ ]:
from transformers import BertTokenizer

# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
# Long text to be split into chunks
long_text = long_txt

# Function to split text into chunks with overlap
def split_text_into_chunks(text, tokenizer, max_length=512, overlap=128):
    tokenized_text = tokenizer.encode(text, add_special_tokens=False)
    chunks = []
    start = 0
    while start < len(tokenized_text):
        end = start + max_length
        chunk = tokenized_text[start:end]
        chunks.append(chunk)
        start += max_length - overlap  # Move the start position ahead with overlap
    return chunks

# Split long text into chunks
chunks = split_text_into_chunks(long_text, tokenizer, max_length=512, overlap=128)

# Print information about the chunks
for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {len(chunk)} tokens")
    print(tokenizer.decode(chunk[:50]))  # Print first 50 tokens for visualization
    print("...")
    print()

Chunk 1: 512 tokens
section 258 of republic act ( r. a. ) no. 7160, also known as the local government code of 1991 ( lgc ), requires the treasurer of the local government to send the warrant of levy to the delinquent owner of
...

Chunk 2: 512 tokens
, ] marikina city. ” sometime in january 1997, transmix builders & construction, inc. ( respondent ), bought the three lots from santos. consequently, the titles under santos ’ name were cancelled, and new titles were issued
...

Chunk 3: 512 tokens
and pay the delinquencies. he claimed that they were unaware of the assessments and statements of account sent by the city treasurer to santos. on july 12, 2010, the city government of antipolo passed city ordinance no. 2010 - 39
...

Chunk 4: 512 tokens
indicated in the tax declarations at that time. thus, on november 17, 2010, respondent updated and settled the rpt due on the three lots as indicated in the notices. the office of the city treasurer issued the corresponding certifications on j